In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
df = pd.read_csv('train.csv')
df.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782


In [19]:
train = df[['effectiveness_rating', 'number_of_times_prescribed',	'base_score']]
train.head()

,effectiveness_rating,number_of_times_prescribed,base_score
0,9,27,8.022969
1,8,192,7.858458
2,5,17,6.341969
3,9,37,6.590176
4,2,43,6.144782


In [20]:
train.corr()

,effectiveness_rating,number_of_times_prescribed,base_score
effectiveness_rating,1.000000,0.242224,0.412301
number_of_times_prescribed,0.242224,1.000000,0.134295
base_score,0.412301,0.134295,1.000000


In [0]:
x = train[['effectiveness_rating',	'number_of_times_prescribed']]
y = train[['base_score']]

In [7]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(x)  # fit does nothing.
transformer

Normalizer(copy=True, norm='l2')

In [0]:
x_norm = transformer.transform(x)

In [92]:
from sklearn import tree
clf = tree.DecisionTreeRegressor(criterion='friedman_mse')
clf.fit(x, y)

DecisionTreeRegressor(ccp_alpha=0.5, criterion='friedman_mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [93]:
clf

DecisionTreeRegressor(ccp_alpha=0.5, criterion='friedman_mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [0]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(x)

In [95]:
y_pred

array([7.50975766, 5.85995262, 5.85995262, ..., 7.50975766, 5.85995262,
       5.85995262])

In [96]:
df_test = pd.read_csv('test.csv')
df_test.head()

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating
0,163740,Mirtazapine,"""I&#039;ve tried a few antidepressants over th...",28-Feb-12,22,Depression,10
1,39293,Contrave,"""Contrave combines drugs that were used for al...",5-Mar-17,35,Weight Loss,9
2,208087,Zyclara,"""4 days in on first 2 weeks. Using on arms an...",3-Jul-14,13,Keratosis,4
3,23295,Methadone,"""Ive been on Methadone for over ten years and ...",18-Oct-16,21,Opiate Withdrawal,7
4,97013,Ambien,"""Ditto on rebound sleepless when discontinued....",13-Jan-15,44,Insomnia,2


In [0]:
predict_on = df_test[['effectiveness_rating', 'number_of_times_prescribed']]

In [0]:
z = clf.predict(predict_on)

In [0]:
submit = pd.DataFrame(columns=['patient_id', 'base_score'])

In [100]:
submit['patient_id'] = df_test['patient_id']
submit.head()

,patient_id,base_score
0,163740,NaN
1,39293,NaN
2,208087,NaN
3,23295,NaN
4,97013,NaN


In [101]:
submit['base_score'] = z
submit.head()

,patient_id,base_score
0,163740,7.509758
1,39293,7.509758
2,208087,5.859953
3,23295,5.859953
4,97013,5.859953


In [0]:
submit.to_csv('dt_fmse_ccp.csv', index=False)

-------------------------------------------------------------------

In [136]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

rfr = xgb.XGBRegressor(max_depth=len(y)) #(max_depth=len(y), random_state=0, criterion='friedman_mse', oob_score=True)
rfr.fit(x, y)

[05:19:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=32165, min_child_weight=1, missing=None,
             n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [0]:
z = rfr.predict(predict_on)

In [138]:
submit = pd.DataFrame(columns=['patient_id', 'base_score'])
submit.head()

,patient_id,base_score


In [139]:
submit['patient_id'] = df_test['patient_id']
submit.head()

,patient_id,base_score
0,163740,NaN
1,39293,NaN
2,208087,NaN
3,23295,NaN
4,97013,NaN


In [140]:
submit['base_score'] = z
submit.head()

,patient_id,base_score
0,163740,8.756915
1,39293,8.744644
2,208087,5.952253
3,23295,5.994741
4,97013,5.370646


In [0]:
submit.to_csv('xgb(1).csv', index=False)